In [1]:
import dm_control.suite as suite
import numpy as np
import torch

# Load the motion capture data
def read_frames_from_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    frames = []
    current_frame = []

    for line in lines:
        if line.strip().isdigit():
            # New frame, save the previous one
            if current_frame:
                frames.append(current_frame)
            current_frame = []
        else:
            # Extract numerical values from each line
            values = [float(val) for val in line.strip().split()[1:]]
            current_frame.extend(values)

    # Add the last frame
    if current_frame:
        frames.append(current_frame)

    return torch.tensor(frames)

file_path = 'experimental/data/0.txt'
data0 = read_frames_from_file(file_path)
file_path = 'experimental/data/1.txt'
data1 = read_frames_from_file(file_path)
file_path = 'experimental/data/2.txt'
data2 = read_frames_from_file(file_path)

min_frames = min(data0.shape[0], data1.shape[0], data2.shape[0])

data = torch.zeros((3, min_frames, data0.shape[1]))
data[0] = data0[torch.linspace(0, data0.shape[0] - 1, min_frames).long()]
data[1] = data1[torch.linspace(0, data1.shape[0] - 1, min_frames).long()]
data[2] = data2[torch.linspace(0, data2.shape[0] - 1, min_frames).long()]

print(data.shape)

torch.Size([3, 698, 62])


In [2]:
num_samples, t_steps, dy = data.shape
dx = 1
batch_size = 1

x = torch.linspace(0, 1, t_steps).repeat(batch_size, 1).view(batch_size, -1, 1)

In [4]:
%env MUJOCO_GL=egl

# Create the humanoid environment
env = suite.load(domain_name="humanoid_CMU", task_name="stand")

# Reset the environment to its initial state
time_step = env.reset()

test_id = 0

# Iterate through the motion capture data
for i in range(t_steps):
    # Apply the motion capture data to the humanoid model

    torso_info = data[test_id, i, :6].numpy()
    joint_info = data[test_id, i, 6:].numpy()

    # Combine torso and joint information into a single action list
    action = np.concatenate([torso_info, joint_info]).tolist()

    # Step the environment
    time_step = env.step(joint_info)

    # You may want to add a delay or control the playback speed here

    # Render the environment
    env.physics.render()

# Close the environment
env.close()

env: MUJOCO_GL=egl
